<a href="https://colab.research.google.com/github/gdebasish11/MalarialCellDetection_CNN/blob/main/MalarialCellDetection_Sequential.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
##Datasource: https://lhncbc.nlm.nih.gov/publication/pub9932

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image
import tensorflow as tf
from tensorflow import keras as tfk

In [9]:
SIZE = 64
dataset = []  
label = []

In [12]:
## define path for parasitized images
parasitized_images = os.listdir("/content/drive/MyDrive/MalarialCellDetection_CNN/train_images/Paratizd_New")

In [13]:
for i, image_name in enumerate(parasitized_images):      
    if (image_name.split('.')[1] == 'png' and i<7000):
        image = cv2.imread('/content/drive/MyDrive/MalarialCellDetection_CNN/train_images/Paratizd_New/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        dataset.append(np.array(image))
        label.append(0)

In [14]:
uninfected_images = os.listdir("/content/drive/MyDrive/MalarialCellDetection_CNN/train_images/Uninfected_New")

In [15]:
for i, image_name in enumerate(uninfected_images):  
    
    if (image_name.split('.')[1] == 'png' and i<7000):
        image = cv2.imread('/content/drive/MyDrive/MalarialCellDetection_CNN/train_images/Uninfected_New/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        dataset.append(np.array(image))
        label.append(1)

# *Sequential model*

In [16]:
input_shape=dataset[1].shape

In [17]:
##Normal sequential model conv2d
inputs = tf.keras.Input(shape=input_shape)
model_seq=tf.keras.Sequential()
model_seq.add(tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape=input_shape))
model_seq.add(tf.keras.layers.MaxPool2D((2,2)))
model_seq.add(tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
model_seq.add(tf.keras.layers.MaxPool2D((2,2)))
model_seq.add(tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
##flatten
model_seq.add(tf.keras.layers.Flatten())
model_seq.add(tf.keras.layers.Dense(64,activation='relu'))
model_seq.add(tf.keras.layers.Dense(2, activation = 'softmax'))

In [11]:
model_seq.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 9216)              0

In [12]:
model_seq.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 9216)              0

In [18]:
# 1. Training data: 80%
# 2. Testing data: 20%
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, tfk.utils.to_categorical(np.array(label)), test_size = 0.1, random_state = 23)

In [19]:
type(X_train),np.array(X_train).shape,y_train.shape,y_test.shape,type(y_test),np.array(X_test).shape

(list, (2069, 64, 64, 3), (2069, 2), (230, 2), numpy.ndarray, (230, 64, 64, 3))

In [15]:
model_seq.compile(optimizer='adam',loss='binary_crossentropy',metrics='accuracy')

In [16]:
history = model_seq.fit(np.array(X_train), y_train, epochs=100,  validation_data=(np.array(X_test), y_test))

Epoch 1/100
65/65 [==============================] - 19s 248ms/step - loss: 4.9124 - accuracy: 0.5230 - val_loss: 0.6911 - val_accuracy: 0.5043
Epoch 2/100
65/65 [==============================] - 15s 235ms/step - loss: 0.6874 - accuracy: 0.5186 - val_loss: 0.6930 - val_accuracy: 0.5087
Epoch 3/100
65/65 [==============================] - 16s 243ms/step - loss: 0.6787 - accuracy: 0.5239 - val_loss: 0.6960 - val_accuracy: 0.5261
Epoch 4/100
65/65 [==============================] - 15s 232ms/step - loss: 0.6814 - accuracy: 0.5452 - val_loss: 0.7031 - val_accuracy: 0.4739
Epoch 5/100
65/65 [==============================] - 16s 244ms/step - loss: 0.6718 - accuracy: 0.5563 - val_loss: 0.7245 - val_accuracy: 0.4957
Epoch 6/100
65/65 [==============================] - 16s 240ms/step - loss: 0.6667 - accuracy: 0.5887 - val_loss: 0.7351 - val_accuracy: 0.5478
Epoch 7/100
65/65 [==============================] - 16s 249ms/step - loss: 0.6459 - accuracy: 0.6201 - val_loss: 0.7387 - val_accuracy:

# *Hyperparameter training with Keras tuner*

In [ ]:
!pip install keras-tuner

In [20]:
def build_tuned_model(hp):
  inputs = tf.keras.Input(shape=input_shape)
  model=tf.keras.Sequential()
  model.add(tf.keras.layers.Conv2D(
          filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
          kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
          activation='relu',input_shape=input_shape
          ))
  model.add(tf.keras.layers.MaxPool2D((2,2)))
  model.add(tf.keras.layers.Conv2D(
          filters=hp.Int('conv_2_filter',min_value=32,max_value=128,step=16),
          kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
          activation='relu'
          ))
  model.add(tf.keras.layers.MaxPool2D((2,2)))
  model.add(tf.keras.layers.Conv2D(
          filters=hp.Int('conv_3_filter',min_value=32,max_value=128,step=16),
          kernel_size=hp.Choice('conv_3_kernel',values=[3,5]),
          activation='relu'
          ))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(
          units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
          activation='relu'
           ))
  model.add(tf.keras.layers.Dense(2, activation = 'softmax'))
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics='accuracy')
  return model

In [3]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-3-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [21]:
tuner_search=RandomSearch(build_tuned_model,
                          objective='val_accuracy',
                          max_trials=5,
                        #  directory='output',
                          project_name="Malarial Cell Detection")

In [22]:
tuner_search.search(np.array(X_train), y_train, epochs=100,  validation_data=(np.array(X_test), y_test))

Trial 5 Complete [00h 00m 58s]
val_accuracy: 0.834782600402832

Best val_accuracy So Far: 0.947826087474823
Total elapsed time: 00h 05m 23s


In [23]:
tuned_model=tuner_search.get_best_models(num_models=1)[0]

In [24]:
tuned_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 48)        1344      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 48)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 112)       134512    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 112)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 11, 11, 64)        64576     
                                                                 
 flatten (Flatten)           (None, 7744)              0